In [1]:
import bs4
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

url = 'https://www.linkedin.com/jobs/view/3331085964/?alternateChannel=search&refId=7U624yHPY2NYK6XeiNch8g%3D%3D&trackingId=iTJP3LR2ymAdSC8wbvFQBw%3D%3D'


def parse_html(html):
    #https://stackoverflow.com/questions/14694482/converting-html-to-text-with-python
    elem = BeautifulSoup(html, features="html.parser")
    text = ''
    for e in elem.descendants:
        if isinstance(e, str):
            text += e.strip()+" "
        elif e.name in ['br',  'p', 'h1', 'h2', 'h3', 'h4','tr', 'th']:
            text += '\n'
        elif e.name == 'li':
            text += '\n- '
    return text

def extract_raw_info(url):
    #https://stackoverflow.com/questions/65973802/scraping-linkedin-job-requirements
    driver = webdriver.Chrome()
    driver.get(url)
    driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/section[2]/div/a/img')
    title = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/section[2]/div/div[1]/div/h1').text
    seniority = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div/section[1]/div/ul/li[1]/span').text
    employment_type = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div/section[1]/div/ul/li[2]/span').text
    job_function = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div/section[1]/div/ul/li[3]/span').text
    company = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/section[2]/div/div[1]/div/h4/div[1]/span[1]/a').text
    location = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/section[2]/div/div[1]/div/h4/div[1]/span[2]').text
    #Find the job description
    job_desc_element = driver.find_element(By.XPATH,'//div[@class="show-more-less-html__markup show-more-less-html__markup--clamp-after-5"]')
    #Get the html of the element and pass into BeautifulSoup parser
    html_job_desc = job_desc_element.get_attribute('outerHTML')
    job_desc = parse_html(html_job_desc)


    driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/section[2]/div/a/img').click()
    time.sleep( 3 )
    driver.find_element(By.XPATH,'//*[@id="contextual-sign-in"]/div/section/button').click()
    time.sleep( 3 )
    company_size = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/section[1]/div/dl/div[3]/dd').text
    specialties = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/section[1]/div/dl/div[7]/dd').text
    company_desc_element = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/section[1]/div/p')
    html_company_desc = company_desc_element.get_attribute('outerHTML')
    company_desc = parse_html(html_company_desc)


    return {"position_details":{"title":title,"seniority":seniority,"employment_type":employment_type,"job_function":job_function,"location":location,"job_desc":job_desc},\
        "company_details":{"company_name":company,"company_size":company_size, "specialties":specialties,"company_desc":company_desc}}


In [2]:
raw_info = extract_raw_info(url)

# QA with deepset/roberta-base-squad2

In [3]:
from transformers import pipeline

model_checkpoint = "deepset/roberta-base-squad2"
question_answerer = pipeline("question-answering", model=model_checkpoint)


c:\Users\pablo\Documents\DS_projects\_aux_files\ds_environment\env\ds_environment\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
All model checkpoint layers were used when initializing TFRobertaForQuestionAnswering.

All the layers of TFRobertaForQuestionAnswering were initialized from the model checkpoint at deepset/roberta-base-squad2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForQuestionAnswering for predictions without further training.


Question 1c: Product or service offered?

In [4]:
question_answerer(question = "What is this company's product?", context = raw_info["company_details"]["company_desc"])

{'score': 0.06878691166639328,
 'start': 1,
 'end': 56,
 'answer': 'Qonto is the leading European business finance solution'}

In [5]:
question_answerer(question = "What has the company created?", context = raw_info["company_details"]["company_desc"])

{'score': 0.00385591690428555,
 'start': 315,
 'end': 333,
 'answer': '900 happy Qontoers'}

In [6]:
question_answerer(question = "What does the company do?", context = raw_info["company_details"]["company_desc"])

{'score': 0.14269354939460754,
 'start': 61,
 'end': 155,
 'answer': 'simplifies everything from everyday banking and financing, to bookkeeping and spend management'}

In [7]:
question_answerer(question = "What is the company doing?", context = raw_info["company_details"]["company_desc"])

{'score': 0.07931458204984665,
 'start': 163,
 'end': 193,
 'answer': 'energizes SMEs and freelancers'}

Question 1d: number of customers or users

In [8]:
question_answerer(question = "Who uses this product?", context = raw_info["company_details"]["company_desc"])

{'score': 0.4971333146095276,
 'start': 173,
 'end': 193,
 'answer': 'SMEs and freelancers'}

In [9]:
question_answerer(question = "How many clients does the company have?", context = raw_info["company_details"]["company_desc"])

{'score': 0.008943535387516022, 'start': 315, 'end': 318, 'answer': '900'}

In [10]:
question_answerer(question = "How many SMEs and freelancers use the product?", context = raw_info["company_details"]["company_desc"])

{'score': 0.00943956058472395, 'start': 361, 'end': 368, 'answer': '300,000'}

In [11]:
question_answerer(question = "How many companies use the product?", context = raw_info["company_details"]["company_desc"])

{'score': 0.5219732522964478, 'start': 361, 'end': 368, 'answer': '300,000'}

In [12]:
question_answerer(question = "How many people use the product?", context = raw_info["company_details"]["company_desc"])

{'score': 0.024822494015097618, 'start': 361, 'end': 368, 'answer': '300,000'}

In [13]:
question_answerer(question = "How many customers use the product?", context = raw_info["company_details"]["company_desc"])

{'score': 0.037519294768571854, 'start': 315, 'end': 318, 'answer': '900'}

Question 1f: The company's position in the market and its growth

In [14]:
question_answerer(question = "How much is the company growing?", context = raw_info["company_details"]["company_desc"])

{'score': 0.00021318308426998556,
 'start': 315,
 'end': 333,
 'answer': '900 happy Qontoers'}

In [15]:
question_answerer(question = "What is the position of this company in the market?", context = raw_info["company_details"]["company_desc"])

{'score': 0.020363856106996536,
 'start': 14,
 'end': 56,
 'answer': 'leading European business finance solution'}

Question 2.a.iii: Team composition

In [16]:
question_answerer(question = "What is the composition of the team?", context = raw_info["position_details"]["job_desc"])

{'score': 0.06472492963075638,
 'start': 5526,
 'end': 5552,
 'answer': '44% of our teams are women'}

In [17]:
question_answerer(question = "Is it a homogeneous team or a multidisciplinary one?", context = raw_info["position_details"]["job_desc"])

{'score': 0.0012040950823575258, 'start': 6194, 'end': 6195, 'answer': '\n'}

Question 2.b.ii: Required education

In [18]:
question_answerer(question = "Which university degree is required?", context = raw_info["position_details"]["job_desc"])

{'score': 0.0020310021936893463,
 'start': 2321,
 'end': 2346,
 'answer': 'Machine Learning Engineer'}

Question 2.b.iii: Technologies

In [19]:
question_answerer(question = "Which programming language is required?", context = raw_info["position_details"]["job_desc"])

{'score': 0.2049625664949417, 'start': 4696, 'end': 4703, 'answer': 'English'}

In [20]:
question_answerer(question = "Which disciplines are used: Statistics, Machine Learning or Deep Learning?", context = raw_info["position_details"]["job_desc"])

{'score': 0.007333592977374792,
 'start': 2120,
 'end': 2136,
 'answer': 'Machine Learning'}

In [21]:
question_answerer(question = "What other technologies are used?", context = raw_info["position_details"]["job_desc"])

{'score': 0.000983073958195746,
 'start': 4237,
 'end': 4273,
 'answer': 'Scikit-learn, Tensorflow, or Pytorch'}

In [22]:
question_answerer(question = "What technologies are used?", context = raw_info["position_details"]["job_desc"])

{'score': 0.0012592421844601631, 'start': 3501, 'end': 3506, 'answer': 'Simon'}

Question 2.b.iv: Required soft skills

In [23]:
question_answerer(question = "What soft skills are required?", context = raw_info["position_details"]["job_desc"])

{'score': 0.0031710718758404255,
 'start': 4496,
 'end': 4509,
 'answer': 'Communication'}

Question 2.b.v: Languages

In [24]:
question_answerer(question = "What languages are required?", context = raw_info["position_details"]["job_desc"])

{'score': 0.18970327079296112, 'start': 4696, 'end': 4703, 'answer': 'English'}

Question 2.c.ii: Remote working and flexibility

In [25]:
question_answerer(question = "Is there a rigid work schedule or flexible working hours?", context = raw_info["position_details"]["job_desc"])

{'score': 0.0015343816485255957,
 'start': 1050,
 'end': 1059,
 'answer': '\nTeamwork'}

In [26]:
question_answerer(question = "Is it an in-office position or a remote position?", context = raw_info["position_details"]["job_desc"])

{'score': 0.014793108217418194,
 'start': 2120,
 'end': 2145,
 'answer': 'Machine Learning Engineer'}

Question 2.c.iii: Salary

In [27]:
question_answerer(question = "What is the salary for this position?", context = raw_info["position_details"]["job_desc"])

{'score': 0.007247203961014748,
 'start': 2321,
 'end': 2346,
 'answer': 'Machine Learning Engineer'}

Question 2.c.iv: Holidays

In [28]:
question_answerer(question = "How many days of paid leave are offered?", context = raw_info["position_details"]["job_desc"])

{'score': 0.15390053391456604, 'start': 5250, 'end': 5252, 'answer': '22'}

Question 2.c.iv: Benefits

In [29]:
question_answerer(question = "Which insurance is included?", context = raw_info["position_details"]["job_desc"])

{'score': 0.14566805958747864,
 'start': 5296,
 'end': 5322,
 'answer': 'Fondo Est health insurance'}

In [30]:
question_answerer(question = "What other benefits does the company offer to its employees?", context = raw_info["position_details"]["job_desc"])

{'score': 0.02142578549683094,
 'start': 3319,
 'end': 3338,
 'answer': 'supporting teamwork'}

# Named entity recognition

In [31]:
import spacy
import pandas as pd

nlp = spacy.load("en_core_web_lg")

In [32]:

doc = nlp(raw_info["position_details"]["job_desc"])
list_ent = []
list_label = []
for ent in doc.ents:
    list_ent.append(ent.text)
    list_label.append(ent.label_)
all_entities = pd.DataFrame({"entity":list_ent,"label":list_label}).drop_duplicates()

In [33]:
all_entities[all_entities["label"].isin(["PERSON","DATE","TIME","CARDINAL","GPE", "PERCENT","LOC","ORDINAL"])==False]

,entity,label
5,SME,ORG
15,Penta,ORG
18,Qontoers,PRODUCT
21,Data Engineers,ORG
22,BI,ORG
23,Data Scientists,ORG
24,Machine Learning Engineers,ORG
26,Data Products,ORG
27,⭐,NORP
32,AI/ML,ORG


In [34]:
all_entities[all_entities["label"].isin(["DATE","TIME","CARDINAL", "PERCENT","ORDINAL"])]

,entity,label
4,July 2017,DATE
7,"300,000",CARDINAL
9,10,CARDINAL
20,55,CARDINAL
33,dozens,CARDINAL
45,one million,CARDINAL
46,2025,DATE
62,first,ORDINAL
68,Monthly,DATE
70,22 days,DATE
